# **최종 모델**

In [ ]:
# 라운지 모델 최종
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from google.colab import drive
drive.mount('/content/drive')


# 사용자Input: 사용자Index(int) / 위치(str) / 시간(datetime)+에 따른 혼잡도 / 사용자 수(int) / 책상높이(1/0) / 콘센트 필요도(1-5) / 컴퓨터 필요도(1-5) / 정수기및화장실유무(1/0) 
user_1 = [17, 'SK미래관', '2019-02-14T16:55:31+00:00', 1, 1, 4, 3, 1] 
theme = 1


# 라운지 데이터
lounge_data = pd.read_csv('/content/라운지 데이터 최종.csv', sep = ",",encoding='EUC-KR') 

# 콘센트 필요도 삽입
lounge_data.insert(8,'의자당 콘센트수', lounge_data['콘센트 수'] / lounge_data['의자 수'])
lounge_data.insert(8,'콘센트 필요도', pd.cut(lounge_data['의자당 콘센트수'], 5))
lounge_data['콘센트 필요도'] = lounge_data['콘센트 필요도'].astype('str')
lounge_data = lounge_data.replace(['(-0.002, 0.4]','(0.4, 0.8]', '(0.8, 1.2]', '(1.2, 1.6]','(1.6, 2.0]'],[1,2,3,4,5]) # 콘센트 필요도 변환

# 테이블당 의자수 삽입
lounge_data.insert(4,'테이블당 의자수', lounge_data['의자 수'] / lounge_data['테이블 수']) #사용자 수 열과의 비교 목적 

# 컴퓨터 필요도 삽입
lounge_data.insert(17,'컴퓨터 필요도', pd.cut(lounge_data['컴퓨터'], 5))
lounge_data['컴퓨터 필요도'] = lounge_data['컴퓨터 필요도'].astype('str')
lounge_data = lounge_data.replace(['(-0.01, 2.0]','(2.0, 4.0]', '(4.0, 6.0]', '(6.0, 8.0]','(8.0, 10.0]'],[1,2,3,4,5]) # 컴퓨터 필요도 변환

# 정수기 및 화장실 유무 삽입
lounge_data.insert(17,'정수기및화장실유무', lounge_data['정수기'] * 0.5 + lounge_data['화장실'] * 0.5)

# 코사인유사도 계산을 위한 최종 라운지 데이터
lounge_data_1 = lounge_data.drop(columns=["lounge_index", "위치(층)", "테이블 수", "의자 수", "냉난방장치", "의자당 콘센트수","콘센트 수", "창문", "컴퓨터", "정수기", "화장실", "혼잡도"]) 



# 코사인유사도 계산 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))


# max min 정규화 함수
def minmax_norm(df_input):
    return df_input.apply(lambda x: (x-x.min())/ (x.max() - x.min()))



# 건물 간 이동시간 데이터
distance = pd.read_csv('/content/drive/MyDrive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물별 이동시간 최종.csv', sep = ",",error_bad_lines=False) 
print(distance.head())


# User Input 기준 이동시간 col 생성 함수
def make_distance(user):
  ls = [] # lounge_data_1의 전체 행에 대한 이동시간 리스트
  temp = distance[distance.building == user[1]]
  temp = temp.reset_index(drop=True)
  print("temp")
  print(temp)
  
  for idx, row in lounge_data_1.iterrows():
    distance_user = temp.loc[0][lounge_data_1.loc[idx]['건물']]
    ls.append([lounge_data_1.loc[idx]['건물'], distance_user])

  temp2 = pd.DataFrame(ls, columns= [ '건물' ,'이동시간'])
  temp2['이동시간'] = (temp2['이동시간'] - temp2['이동시간'].min()) / ( temp2['이동시간'].max() - temp2['이동시간'].min())
  print("temp2") 
  print(temp2)

  lounge_data_2 = pd.merge(lounge_data_1, temp2, left_index=True, right_index=True, how='right')

  print("lounge_data_2")
  print(lounge_data_2)
  return lounge_data_2



# 코사인유사도 산출 외 가중치용 라운지 데이터
lounge_data_3 = lounge_data.drop(columns=["lounge_index" ,"라운지", "위치(층)","테이블당 의자수", "테이블 수", "의자당 콘센트수","콘센트 수", "컴퓨터", "정수기", "화장실", "컴퓨터 필요도"]) 
print(lounge_data_3)




# 혼잡도 데이터
con_0 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_월.csv')
con_1 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_화.csv')
con_2 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_수.csv')
con_3 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_목.csv')
con_4 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_금.csv') 

day = str(pd.to_datetime(user_1[2].split('T')[0]).dayofweek)
user_c = con_0

#find day
if day==1:
  user_c = con_1
elif day==2:
  user_c = con_2
elif day==3:
  user_c = con_3
elif day==4:
  user_c = con_4

range = pd.date_range("09:00", "23:30", freq="30min").strftime('%H:%M:%S')
dt = pd.to_datetime(user_1[2]).round('30min')
user_c_r = range.get_loc(str(dt).split(' ')[1].split('+')[0])
lounge_data_3['혼잡도']=lounge_data_3.apply(lambda row: user_c[row.건물][user_c_r], axis=1)



# 테마별 (유사도 자체에 대한 )가중치 적용 함수
def theme_score(weight, theme, idx):

   #개인공부
    if theme == 0: 
      # 혼잡도
      weight[1] *= (1 + (1 - lounge_data_3.loc[idx]['혼잡도']) * 0.5)
   #휴식
    if theme == 1: 
      # 책상 높이
      if lounge_data_3.loc[idx]['책상 높이'] == 0: 
        weight[1] *= 1.71
      else:
        weight[1] *= 1.15

      # 정수기 및 화장실 유무
      weight[1] *= (1 + (1 - lounge_data_3.loc[idx]['정수기및화장실유무']))

      # 혼잡도
      weight[1] *= (1 + (1 - lounge_data_3.loc[idx]['혼잡도']) * 0.3)
   #대화
    if theme == 2: 
      # 혼잡도
      weight[1] *= (1 + (1 - lounge_data_3.loc[idx]['혼잡도']) * 0.1)
   #팀공부(팀플)
    if theme == 3: 
      # 혼잡도
      weight[1] *= (1 + (1 - lounge_data_3.loc[idx]['혼잡도']) * 0.1)

    return weight




# 최종 추천 함수
def recommendation(user_input, theme):


  lounge_org = make_distance(user_input)
  lounge_org = lounge_org.rename(columns={'건물_x':'건물'})
  lounge_org = lounge_org.drop(columns=["건물_y"])
  lounge_0 = lounge_org.drop(columns=["건물", "라운지"])
  lounge = lounge_0.drop(columns=["이동시간"])
  del user_input[0:3]


  # 사용자 input - 라운지 data 간의 공간 점수 계산 및 내림차순 정렬
  result = [] # 모든 라운지에 대한 공간 (코사인유사도) 점수 값 저장 리스트

  for idx, row in lounge.iterrows():
   
    # (테마와 무관한 공통 코사인유사도)
    row = row.values.tolist()
    similarity_org = [idx, cos_sim(user_input, row)] 
    similarity_weight = similarity_org


    # 거리에 대한 가중치 부여 (테마와 무관한 공통 가중치 1)
    similarity_weight[1] *= (1 - lounge_0.loc[idx]['이동시간']) * 2


    # 의자수 > 사용자 수 가중치 부여 (=필터링 / 테마와 무관한 공통 가중치 2)
    if lounge_data_3.loc[idx]['의자 수'] < user_input[0]:
      similarity_weight[1] *= 0
    else: pass


    # 에어컨 유뮤 / 창문 유무에 대한 가중치 부여 (테마와 무관한 공통 가중치 3)
    # 에어컨 유무
    if lounge_data_3.loc[idx]['냉난방장치'] == 1:
      similarity_weight[1] *= (1 + 0.5)
    else: pass


    # 창문 유무
    if lounge_data_3.loc[idx]['창문'] == 1:
      similarity_weight[1] *= (1 + 0.1)
    else: pass


    # 테마별 가중치 적용
    similarity_weight = theme_score(similarity_weight, theme, idx)

    result.append(similarity_weight)  # 결과값 리스트로 저장
  
  result_df = pd.DataFrame(result,columns= ['idx', 'CosineSimilarity_weight']) 
  result_df = result_df.sort_values(by='CosineSimilarity_weight' ,ascending=False) # 내림차순 정렬
  final_df =  pd.merge(lounge_org, result_df, left_index=True, right_index=True, how='right') #result_df = pd.merge(result_df, top18_org, on= 'idx', how ='left') 같은 방법
  final_df = final_df.iloc[1:10] # 상위 9개의 라운지 추출 
  final_df =  final_df[['건물', '라운지']].values
 
  return final_df

print(recommendation(user_1, theme)) # userInput 과 themeNum




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     building    본관  중앙도서관(대학원)  ...  안암인터내셔널하우스  프런티어관(G동)  안암글로벌하우스
0          본관   0.0        38.0  ...       155.4      154.1     155.4
1  중앙도서관(대학원)  38.0         0.0  ...       123.4      122.1     123.4
2       사색의동산  92.7        72.7  ...       128.9      127.6     128.9
3        다람쥐길   4.1        42.1  ...       159.5      158.2     159.5
4   동원글로벌리더십홀  53.7        15.7  ...       112.0      110.7     112.0

[5 rows x 99 columns]
           건물  책상 높이  의자 수  냉난방장치  콘센트 필요도  창문  혼잡도  정수기및화장실유무
0     4.18기념관      1     6      0        2   1  NaN        0.5
1     4.18기념관      1     2      0        5   1  NaN        1.0
2     4.18기념관      1    30      1        1   0  NaN        0.0
3     4.18기념관      1    24      0        1   0  NaN        0.5
4       CJ법학관      1    14      1        1   1  NaN        1.0
..        ...    ...   ...    ...      ...  ..  ..